In [21]:
#pip install tensorflow pandas scikit-learn

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
# Step 1: Load the dataset
file_path = r"C:\Users\theaa\Downloads\final_data18k.csv"  # Adjust the path as necessary
data = pd.read_csv(file_path)

In [6]:
# Step 2: Clean the data
cleaned_data = data.dropna(subset=['Comment'])  # Drop rows with missing text
X = cleaned_data['Comment']
y = cleaned_data['label']

In [7]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# Step 3: Preprocess the text data
max_words = 10000  # Maximum vocabulary size
max_len = 100      # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='pre')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='pre')

In [9]:
# Step 4: Build the LSTM model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# Step 5: Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=10,
    batch_size=32
)

Epoch 1/10
407/407 [==============================] - 39s 89ms/step - loss: 0.4276 - accuracy: 0.7951 - val_loss: 0.3120 - val_accuracy: 0.8742
Epoch 2/10
407/407 [==============================] - 36s 88ms/step - loss: 0.2487 - accuracy: 0.8978 - val_loss: 0.3127 - val_accuracy: 0.8683
Epoch 3/10
407/407 [==============================] - 34s 83ms/step - loss: 0.1785 - accuracy: 0.9282 - val_loss: 0.3563 - val_accuracy: 0.8658
Epoch 4/10
407/407 [==============================] - 37s 92ms/step - loss: 0.1298 - accuracy: 0.9491 - val_loss: 0.4155 - val_accuracy: 0.8518
Epoch 5/10
407/407 [==============================] - 36s 88ms/step - loss: 0.0933 - accuracy: 0.9632 - val_loss: 0.5193 - val_accuracy: 0.8575
Epoch 6/10
407/407 [==============================] - 34s 85ms/step - loss: 0.0758 - accuracy: 0.9710 - val_loss: 0.5636 - val_accuracy: 0.8507
Epoch 7/10
407/407 [==============================] - 36s 87ms/step - loss: 0.0651 - accuracy: 0.9751 - val_loss: 0.6398 - val_accuracy:

In [12]:
# Step 6: Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

175/175 [==============================] - 4s 24ms/step - loss: 0.8173 - accuracy: 0.8455
Test Loss: 0.8173
Test Accuracy: 0.8455


In [15]:
# Save the trained LSTM model
model.save('cyberbullying_model.keras')

# Save the tokenizer used for preprocessing
import pickle
with open("tokenizer.pkl", "wb") as file:
    pickle.dump(tokenizer, file)